# Example 2 - Synchronizing qcodes database
In this tutorial we add a qcodes database to the QHarbor sync sources and run some live measurements

Before getting started: 
- open the QHarbor sync manager running on terminal: `python -c "import qdrive; qdrive.launch_GUI()"`
- open the dataqruiser app
- esnure you have access to a scope
- connect this notebook to the kernel where qdrive is installed

You can download this jupyter notebook and more examples from [here](https://github.com/qEncoder/dataQruiser/tree/main/examples).

## Qcodes setup

In [1]:
import qcodes as qc
from qcodes.dataset import initialise_database, initialise_or_create_database_at, load_or_create_experiment, LinSweep, dond
from qcodes.instrument_drivers.mock_instruments import DummyInstrument, DummyInstrumentWithMeasurement
import os

os.makedirs('qcodes_data', exist_ok=True)
initialise_or_create_database_at("./qcodes_data/QCoDeS_example.db")
exp = load_or_create_experiment(
    experiment_name='mock_exp',
    sample_name="s001",
)

try: 
    # Creating dummy instruments
    dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
    dmm = DummyInstrumentWithMeasurement('dmm', setter_instr=dac)

    #  Adding instruments to the station
    station = qc.Station()
    station.add_component(dac)
    station.add_component(dmm)
except: pass

## Add the qcodes database as QHarbor sync sources
- open the QHarbor sync manager
- in the Syncronization tab, click on button `Start sync` (if necessary)
- click on button `add source` > select tab `qcodes` > fill in the form and add the database

## Run a qcodes measurement
Run the measurement below. You will see the live dataset streaming in the dataqruiser app (ensure you select the same scope you assigned for syncronizing this database). 

In [2]:
# Setting up a doNd measurement
sweep_1 = LinSweep(dac.ch1, -1, 1, 20, 0.01)
sweep_2 = LinSweep(dac.ch2, -1, 1, 20, 0.02)

ds = dond(
    sweep_1,                          # 1st independent parameter
    sweep_2,                          # 2nd independent parameter
    dmm.v1,                           # 1st dependent parameter
    dmm.v2,                           # 2nd dependent parameter
    measurement_name="qc_sweep_example",  # Set the measurement name
    exp=exp,                          # Set the experiment to save data to.
    show_progress=True,               # Optional progress bar
    write_period=0.1,                 # Ensure short writing period for live plotting
)[0]

Starting experimental run with id: 9. Using 'qcodes.dataset.dond'


  0%|          | 0/400 [00:00<?, ?it/s]

## Import the data from QHarbor
In the dataQruiser app you can use the copy button to copy the python import (or uuid) of a dataset, then paste it here below

In [3]:
# replace this code with your dataset import statement, or use the code below to fetch the dataset from qcodes GUID

from qdrive import dataset
dsq = dataset(ds.guid)
dsx = dsq['measurement'].xarray 

100%|██████████| 41.2k/41.2k [00:00<00:00, 3.09MiB/s]


In [4]:
# Review the content of the dataset
dsq

Contents of dataset :: qc_sweep_example

uuid :: d25a3c64-a2f1-4d84-b52b-2dae1faddd33
Alternative identifier :: 6aa792df-0000-0000-0000-01926660b6a1
Description :: database : QCoDeS_example.db | run ID : 9 | GUID : 6aa792df-0000-0000-0000-01926660b6a1
Scope :: z_scope
Ranking :: 0
Data Indentifiers :: 
	sample : s001
	experiment name : mock_exp
	set-up : test
Files :: 
name         type                  selected version number (version_id)      Maximal version number
-----------  --------------------  --------------------------------------  ------------------------
measurement  FileType.HDF5_NETCDF  0 (1728294467236)                                              0

In [5]:
# Review the content of the measurement file
dsx

<xarray.Dataset> Size: 7kB
Dimensions:  (dac_ch1: 20, dac_ch2: 20)
Coordinates:
  * dac_ch1  (dac_ch1) float64 160B -1.0 -0.8947 -0.7895 ... 0.7895 0.8947 1.0
  * dac_ch2  (dac_ch2) float64 160B -1.0 -0.8947 -0.7895 ... 0.7895 0.8947 1.0
Data variables:
    dmm_v1   (dac_ch1, dac_ch2) float64 3kB ...
    dmm_v2   (dac_ch1, dac_ch2) float64 3kB ...
Attributes: (12/14)
    ds_name:                  qc_sweep_example
    sample_name:              s001
    exp_name:                 mock_exp
    snapshot:                 {'station': {'instruments': {'dac': {'functions...
    guid:                     6aa792df-0000-0000-0000-01926660b6a1
    run_timestamp:            2024-10-07 11:47:47
    ...                       ...
    captured_counter:         9
    run_id:                   9
    run_description:          {'version': 3, 'interdependencies': {'paramspec...
    parent_dataset_links:     []
    run_timestamp_raw:        1728294467.236396
    completed_timestamp_raw:  1728294477.1800792

## Document
To link this dataset in you documentation process you can use:  
- uuid (this is automatically assigned from QHarbor)
- alt_uid (this is the qcodes GUID)
- https hyperlink (when clicked it opens this dataset in the dataqruiser app, contains uuid in the url)

All these info can be copied to clipboard from the dataQruiser app using the `Copy` button.

- You can also take a plot screenshot using the `Save image to clipboard` button located under each plot of the dataQruiser app (this screenshot contains the picture and a text field of all the above information, if you Ctrl click the image it open the dataset in dataqruiser)